In [28]:

from dotenv import load_dotenv, find_dotenv 
load_dotenv()

import pandas as pd
import polars as pl

from financialtools.utils import *

In [29]:
ticker = 'PRY.MI'

metrics, composite_scores, red_flags = get_fin_data(ticker)

In [30]:
metrics

'"[{\\"ticker\\":\\"PRY.MI\\",\\"time\\":2020,\\"GrossMargin\\":null,\\"OperatingMargin\\":null,\\"NetProfitMargin\\":null,\\"EBITDAMargin\\":null,\\"ROA\\":null,\\"ROE\\":null,\\"FCFToRevenue\\":null,\\"FCFYield\\":null,\\"FCFtoDebt\\":null,\\"DebtToEquity\\":null,\\"CurrentRatio\\":null},{\\"ticker\\":\\"PRY.MI\\",\\"time\\":2021,\\"GrossMargin\\":0.32,\\"OperatingMargin\\":0.03,\\"NetProfitMargin\\":0.02,\\"EBITDAMargin\\":0.07,\\"ROA\\":0.03,\\"ROE\\":0.11,\\"FCFToRevenue\\":0.04,\\"FCFYield\\":0.09,\\"FCFtoDebt\\":0.13,\\"DebtToEquity\\":1.28,\\"CurrentRatio\\":0.09},{\\"ticker\\":\\"PRY.MI\\",\\"time\\":2022,\\"GrossMargin\\":0.34,\\"OperatingMargin\\":0.05,\\"NetProfitMargin\\":0.03,\\"EBITDAMargin\\":0.07,\\"ROA\\":0.04,\\"ROE\\":0.14,\\"FCFToRevenue\\":0.04,\\"FCFYield\\":0.09,\\"FCFtoDebt\\":0.19,\\"DebtToEquity\\":0.86,\\"CurrentRatio\\":0.13},{\\"ticker\\":\\"PRY.MI\\",\\"time\\":2023,\\"GrossMargin\\":0.37,\\"OperatingMargin\\":0.07,\\"NetProfitMargin\\":0.03,\\"EBITDAMarg

In [22]:
composite_scores

'"[{\\"ticker\\":\\"PRY.MI\\",\\"time\\":2020,\\"composite_score\\":3.0},{\\"ticker\\":\\"PRY.MI\\",\\"time\\":2021,\\"composite_score\\":2.14},{\\"ticker\\":\\"PRY.MI\\",\\"time\\":2022,\\"composite_score\\":2.46},{\\"ticker\\":\\"PRY.MI\\",\\"time\\":2023,\\"composite_score\\":2.66},{\\"ticker\\":\\"PRY.MI\\",\\"time\\":2024,\\"composite_score\\":2.54}]"'

In [ ]:
red_flags

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict, Optional
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Pydantic output model
class StockRegimeAssessment(BaseModel):
    regime: Literal["bull", "bear", "postpone"] = Field(
        ..., description="The fundamental regime classification of the stock"
    )
    rationale: str = Field(
        ..., description="Concise explanation justifying the regime classification based on the financial metrics, composite ratio and red flags"
    )
    metrics_movement: str = Field(
        ..., description=(
            "A summary description of how key financial metrics have moved across years, "
            "e.g., 'GrossMargin increased steadily, DebtToEquity rose sharply, FCFYield remained stable.'"
        )
    )
    non_aligned_findings: Optional[str] = Field(
        None,
        description=(
            "Observations or signals that are not aligned with the overall metric trends, "
            "such as contradictory indicators, anomalies, or important red flags."
        )
    )


# Instantiate the LLM (OpenAI GPT-4 or your preferred model)
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# Instantiate the parser with the Pydantic model
parser = PydanticOutputParser(pydantic_object=StockRegimeAssessment)
# Wrap your parser with OutputFixingParser
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# Get the format instructions string from the parser
format_instructions = parser.get_format_instructions()

system_prompt_template = """
You are a trader assistant specializing in fundamental analysis. 

Based on the following financial data, provide a concise overall assessment that classifies 
the stock’s current fundamental regime as one of:

- bull: Strong and improving fundamentals supporting a positive outlook.
- bear: Weak or deteriorating fundamentals indicating risk or decline.
- postpone: Mixed or inconclusive fundamentals suggesting to wait for clearer signals.

Financial data constists of financial metrics, composite score and red flags.

Profitability and Margin Metrics:
    -GrossMargin: gross profit / total revenue 
    -OperatingMargin: operating income / total revenue
    -NetProfitMargin: net income / total revenue
    -EBITDAMargin: ebitda / total revenue
Returns metrics:
    -ROA: net income / total assets
    -ROE: net income / total equity
Cash Flow Strength metrics: 
    -FCFToRevenue: free cash flow / total revenue
    -FCFYield: free cash flow / market capitalization
    -FCFToDebt:: free cash flow / total debt
Leverage & Solvency metrics:
    -DebtToEquity: total debt / total equity
Liquidity metrics:
    -CurrentRatio: working capital / total liabilities

The composite score is a weighted average (1 to 5) that summarizes the company’s overall fundamental health.
It reflects profitability, efficiency, leverage, liquidity, and cash flow strength, based on the above mentioned financial metrics.

Range:
1 = Weak fundamentals
5 = Strong fundamentals

Each metric is scored on a 1–5 scale and multiplied by its weight. The composite score is the sum of weighted scores divided by the total weight.

A red flag is an early warning signal that highlights potential weaknesses in a company’s financial statements 
or business quality. These warnings do not always mean immediate distress, but they indicate heightened risk that 
traders should carefully consider before taking a position.

"""

# Create a ChatPromptTemplate with system message and user input

system_prompt_filled = system_prompt_template.format(format_instructions=format_instructions)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_filled),
    ("human", "Metrics:\n{metrics}\nScores:\n{scores}\nRedFlags:\n{red_flags}"),
])

# Create a runnable chain: prompt followed by LLM invocation
chain = prompt | llm | parser


# Then invoke with a dict containing 'financial_data'
response = chain.invoke({
    "metrics": metrics,  
    "scores": composite_scores,
    "red_flags": red_flags,    
})


StockRegimeAssessment(regime='postpone', rationale="Financial metrics show weak to moderate fundamentals with no clear signs of consistent improvement. Composite scores fluctuate around the mid-2s, indicating limited strength. No red flags are present, but vulnerabilities in liquidity and leverage suggest caution. Therefore, the regime is classified as 'postpone' to wait for clearer positive signals.", metrics_movement='Profitability, cash flow, and leverage metrics have shown modest and inconsistent changes over the years, with no sustained upward trend. Composite scores remain around the mid-2s, reflecting weak to moderate fundamentals.', non_aligned_findings=None)

In [21]:
import rich

rich.print(response)

StockRegimeAssessment(
    regime='postpone',
    rationale="Financial metrics show weak to moderate fundamentals with no clear signs of consistent improvement. 
Composite scores fluctuate around the mid-2s, indicating limited strength. No red flags are present, but 
vulnerabilities in liquidity and leverage suggest caution. Therefore, the regime is classified as 'postpone' to 
wait for clearer positive signals.",
    metrics_movement='Profitability, cash flow, and leverage metrics have shown modest and inconsistent changes 
over the years, with no sustained upward trend. Composite scores remain around the mid-2s, reflecting weak to 
moderate fundamentals.',
    non_aligned_findings=None
)